In [13]:
!pip install gurobipy  # install gurobipy, if not already installed

In [14]:
from ast import Pass
from re import M
from turtle import shape
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [15]:
def complete_graph(graph):
        graph_original = graph
        for e in graph_original.edges():
            graph_original[e[0]][e[1]]['length'] = int(graph_original[e[0]][e[1]]['length'])
        graph_complete = graph_original.copy()
        nodes = list(graph_original.nodes())
        paths = {}
        paths = dict(nx.all_pairs_dijkstra_path(graph_original, weight = 'length'))
        for i in nodes:
            for j in nodes:
                if i != j and not (i, j) in graph_complete.edges():
                    graph_complete.add_edge(i, j)
                    graph_complete[i][j]['name'] = '{}to{}'.format(i, j)
                    graph_complete[i][j]['length'] = 0
                    paths[(i, j)] = paths[i][j]
                    for k in range(len(paths[i, j]) - 1):
                        graph_complete[i][j]['length'] += graph_original[paths[i][j][k]][paths[i][j][k + 1]]['length']
                elif i != j:
                    paths[(i, j)] = [i, j]

        return graph_complete

In [17]:
class mid_mile:

    def __init__(self, K, L, M, R, N):

        #self.graph = graph
        self.K = K
        self.L = L                                       # no of legs
        self.M = M
        self.R = R
        self.N = N                                       # no of nodes/warehouses

        self.V = np.zeros(shape=(self.K))           # the demand of each commodity K in route R
        self.v = np.zeros(shape=(self.N, self.N, self.M))

        self.A = np.zeros(shape=(self.N, self.N, self.M))
        self.F = np.zeros(shape=(self.N, self.N, self.M))
        self.B = np.zeros(shape=(self.N, self.N, self.M))


        self.C = np.zeros(shape=(self.R))
        self.matrix = np.zeros(shape=(self.N, self.N, self.R))

        self.model = gp.Model()
        
    def add_variables(self):
        f_list = []
        y_list = []
        rk = []

        # decision variables: xr, ylm, vlm, flm
        for n1 in range(self.N):
            for n2 in range(self.N):
                for m in range(self.M):
                    for r in range(self.R):
                        y_list.append((n1, n2, m, r))
                    f_list.append((n1, n2, m))

        for r in range(self.R):
            for k in range(self.K):
                rk.append((r,k))

        f_list = gp.tuplelist(f_list)
        y_list = gp.tuplelist(y_list)
        rk = gp.tuplelist(rk)

        self.x = self.model.addVars(rk, vtype=GRB.BINARY, name="x")
        self.y = self.model.addVars(y_list, vtype=GRB.BINARY, name="y")
        self.f = self.model.addVars(f_list, vtype=GRB.INTEGER, name="f")

        self.model.update()

    def add_init_constraints(self):
        self.add_constraints1()
        self.add_constraints2()
        self.add_constraints3()
        self.add_constraints4()
        self.add_constraints5()
        #self.obj_constraint()

    def add_constraints1(self):
        # each commodity can have only one route associated
        lhs = 0
        for k in range(self.K):
            for r in range(self.R):
                lhs += self.x[r, k]
            self.model.addConstr(lhs == 1)

    def add_constraints2(self):
        # summation of shipments should tally
        lhs = 0
        for k in range(self.K):
            lhs += self.V[k]
        
        print("--- Constraint 2 running: lhs = {} ---".format(lhs))
        
        rhs = 0
        for n1 in range(self.N):
            for n2 in range(self.N):
                for m in range(self.M):
                    for r in range(self.R):
                        rhs += self.v[n1][n2][m] * self.f[n1, n2, m] * self.y[n1, n2, m, r]

            
        self.model.addConstr(lhs - rhs <= 0)
        
        self.model.update()

    def add_constraints3(self):
        # each route can have only commodity
        ####################### =======================>>>>>>>>>>>>>>>> atmost one commodity ########
        lhs = 0
        for r in range(self.R):
            for k in range(self.K):
                lhs += self.x[r, k]
            self.model.addConstr(lhs <= 1)
        

    def add_constraints4(self):
        # constraint y[l, m]

        # for r in range(self.R):
        #     # rhs = 0
        #     # for k in range(self.K):
        #     #     rhs += self.x[r, k]

            for r in range(self.R):
                for n1 in range(self.N):
                    for n2 in range(self.N):
                        for m in range(self.M):    
                            rhs = 0
                            for k in range(self.K):
                                rhs += self.x[r, k]
                            
                            self.model.addConstr(self.y[n1, n2, m, r] == rhs * self.matrix[n1][n2][r])
    
    def add_constraints5(self):
        for n1 in range(self.N):
            for n2 in range(self.N):
                for m in range(self.M):
                    lhs = self.f[n1, n2, m]
                    #rhs = self.F[n1][n2][m] * self.y[l, m]
                    self.model.addConstr(lhs - 10 <= 0)
            
        self.model.update()


    # def obj_constraint(self):
    #     lhs = 0
    #     for e in self.edges:
    #         lhs += self.graph[e[0]][e[1]]['length'] * self.x[a, e[0], e[1]]
    #     #this constraint ensures that c is atleast equal to the max of the walks of the agents
    #     print(self.c)
    #     self.model.addConstr(lhs <= self.c[0], name="obj_agent{}".format(a))
        
    #     self.model.update()


    def objective(self):
        term2 = 0
        for n1 in range(self.N):
            for n2 in range(self.N):
                for m in range(self.M):
                    term2 += self.A[n1][n2][m]*self.f[n1, n2, m]
        
        term1 = 0
        for k in range(self.K):
            for r in range(self.R):
                term1 += self.x[r, k] * self.C[r]
        
        obj = term1 + term2

        self.model.setObjective(obj, GRB.MINIMIZE)

    def process(self):
        self.status = self.model.optimize()

    def get_solution(self):
        Pass

In [21]:
# Input model parameter values for 3 nodes

K = 1
M = 1                       # only one mode of transport
N = 3
R = 2
L = 3

V = np.zeros(shape=(K))
V[0] = 20

C = [10, 5]

v = np.zeros(shape=(N, N, M))
v[0][1][0] = 2
v[1][2][0] = 1
v[0][2][0] = 4


F = np.zeros(shape=(N, N, M))
F[0][1][0] = 5
F[1][2][0] = 3
F[0][2][0] = 4

A = np.zeros(shape=(N, N, M))
A[0][1][0] = 9
A[1][2][0] = 2
A[0][2][0] = 6

graph = np.zeros(shape=(N, N, R))
graph[0][1][0] = 1
graph[1][2][0] = 1
graph[0][2][1] = 1

# F = np.zeros(shape=(L, M))
# for i in range(L):
#     for j in range(M):
#         F[i][j] = 5*i + j

# A = np.zeros(shape=(L, M))
# for i in range(L):
#     for j in range(M):
#         A[i][j] = 2*(i+1) + 3*(j+1)

In [7]:
print(V)

[20.]


In [8]:
# Input model parameter values for 2 nodes
# for 2 nodes

K = 1
M = 1                       # only one mode of transport
N = 2
R = 1
L = 1

V = np.zeros(shape=(K))
V[0] = 20

C = [10]

v = np.zeros(shape=(N, N, M))
v[0][1][0] = 5


F = np.zeros(shape=(N, N, M))
F[0][1][0] = 9


A = np.zeros(shape=(N, N, M))
A[0][1][0] = 9


graph = np.zeros(shape=(N, N, R))
graph[0][1][0] = 1


# F = np.zeros(shape=(L, M))
# for i in range(L):
#     for j in range(M):
#         F[i][j] = 5*i + j

# A = np.zeros(shape=(L, M))
# for i in range(L):
#     for j in range(M):
#         A[i][j] = 2*(i+1) + 3*(j+1)

In [24]:
test = mid_mile(K, L, M, R, N)

test.V = V
test.v = v
test.F = F
test.A = A
test.C = C
test.matrix = graph

In [25]:
test.add_variables()
test.add_init_constraints()
test.objective()
test.process()

--- Constraint 2 running: lhs = 20.0 ---
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 30 rows, 29 columns and 35 nonzeros
Model fingerprint: 0x6676b319
Model has 1 quadratic constraint
Variable types: 0 continuous, 29 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 4e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [2e+01, 2e+01]
Presolve removed 30 rows and 29 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 35 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.500000000000e+01, best bound 3.500000000000e+01, gap 0.0000%


In [26]:
y_sol = test.model.getAttr('X', test.y)
f_sol = test.model.getAttr('X', test.f)
x_sol = test.model.getAttr('X', test.x)

print(y_sol)

{(0, 0, 0, 0): 0.0, (0, 0, 0, 1): 0.0, (0, 1, 0, 0): 0.0, (0, 1, 0, 1): 0.0, (0, 2, 0, 0): 0.0, (0, 2, 0, 1): 1.0, (1, 0, 0, 0): 0.0, (1, 0, 0, 1): 0.0, (1, 1, 0, 0): 0.0, (1, 1, 0, 1): 0.0, (1, 2, 0, 0): 0.0, (1, 2, 0, 1): 0.0, (2, 0, 0, 0): 0.0, (2, 0, 0, 1): 0.0, (2, 1, 0, 0): 0.0, (2, 1, 0, 1): 0.0, (2, 2, 0, 0): 0.0, (2, 2, 0, 1): 0.0}


In [27]:
y_sol = test.model.getAttr('X', test.y)
f_sol = test.model.getAttr('X', test.f)
x_sol = test.model.getAttr('X', test.x)

print(x_sol)
print(f_sol)

{(0, 0): 0.0, (1, 0): 1.0}
{(0, 0, 0): -0.0, (0, 1, 0): 0.0, (0, 2, 0): 5.0, (1, 0, 0): -0.0, (1, 1, 0): -0.0, (1, 2, 0): 0.0, (2, 0, 0): -0.0, (2, 1, 0): -0.0, (2, 2, 0): -0.0}
